In [90]:
from vapp.package_manager import PackageManager
from vapp.types import ICommand, IContextObject, IFrameData, VApp


In [91]:
%run gptautomator/chat_gpt_automator.py
%run gptautomator/chat_gpt_session.py
from gptautomator.chat_gpt_chrome import ChatGPTChrome


In [92]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER,HREF,XPATH
0,iframe,,humanThirdPartyIframe,,,,,,XPATH: /HTML/BODY/IFRAME[2]
1,iframe,,humanThirdPartyIframe,,,,,,XPATH: /HTML/BODY/IFRAME[2]



In [140]:
specific_page_data = chrome_vapp.get_frame_data("specific_elements")
print(specific_page_data)

ELEMENT_INDEX,TAG,TEXT,ID,NAME,INPUT_TYPE,VALUE,PLACEHOLDER,HREF,XPATH
0,input,,,,text,,Search,,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/DIV/DIV/DIV/INPUT
1,input,,msg-overlay-list-bubble-search__search-typeahead-input,,text,,Search messages,,XPATH: /HTML/BODY/DIV[4]/DIV[4]/ASIDE/DIV/DIV/DIV/INPUT
0,a,,,,,,,https://www.linkedin.com/feed/?doFeedRefresh=true&nis=true,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/A
1,a,Home,,,,,,https://www.linkedin.com/feed/?doFeedRefresh=true&nis=true&,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/NAV/UL/LI/A
2,a,"2
2 new network updates notifications
My Network",,,,,,https://www.linkedin.com/mynetwork/?,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/NAV/UL/LI[2]/A
3,a,Jobs,,,,,,https://www.linkedin.com/jobs/?,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/NAV/UL/LI[3]/A
4,a,Messaging,,,,,,https://www.linkedin.com/messaging/?,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV/NAV/UL/LI[4]/A
5,a,"14
14 new notifications
Notifications",,,,,,https://www.linkedin.com/notifications/?,XPATH: /HTML/BODY/DIV[4]/HEADER/DIV

In [127]:
page_text = chrome_vapp.get_frame_data("body_text")
text_without_empty_lines = '\n'.join(line for line in page_text.splitlines() if line.strip())
page_cleaned_text = text_without_empty_lines

In [ ]:
page_text = chrome_vapp.get_frame_data

In [114]:
chat_gpt = ChatGPTChrome()

Closing the browser...
You need to manually complete the log-in or the human verification if required.



DevTools listening on ws://127.0.0.1:52228/devtools/browser/54d11fa0-ed73-479c-a670-8ec35ae7c02f
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Continuing with the automation process...


In [77]:
chat_gpt.quit()

Closing the browser...


In [115]:
from gptautomator.chat_gpt_session import ChatGPTSession
gptSession = ChatGPTSession(chat_gpt)

In [129]:
import json
prompt = f"""
You will be acting as chrome operating agent.I will give you task,
web page information called frame data and my information required to fill any form or may be my login credentials. 
What you will be sending me is commands. Let me explain

I designed a platform where vapps are installed.
Vapp is nothing but a virtual application that holds programmatical access to applications like chrome, file changing.
My Platform has only one vapp which is chrome. I will give you list of commands available in this app.
Each command is list of values as simple as it is. 
First argument will be command name. Remaining are command related arguments. Ofcourse you will have commands information.
Infact if any command is added, I will be letting you know like below

NEW COMMAND ALERT
    ...Command details...

Installed Vapp and Commands Presently
Vapp : Chrome, id : chrome

Command : Open Url
Response Examples
["open_url","https://linkedin.com"]
["open_url","https://www.ibm.com"]

Commands : Send Keys to the input
Response Examples
["send_keys","/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV/DIV/DIV/INPUT","sarath.devarakonda22@gmail.com"],
["send_keys","/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV[2]/DIV/DIV/INPUT","412,./dec"],

Command: Sleep Command
Response Examples:
["sleep",1]
["sleep", 2]

Command: Click command
Response Examples
["click","/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV[2]/BUTTON""]
["click","/HTML/BODY/BUTTON]

When you have to give commands you need to give like below
{
    json.dumps({
        "understood" : True,
        "commands" : [
            ["open_url","https://example.org"],
            ["send_keys","/HTML/BODY/DIV[2]/INPUT"],
            ["send_keys","/HTML/BODY/DIV[3]/INPUT"],
            ["click","/HTML/BODY/DIV[3]/button"]
        ]
    })
}
YUO ALWAYS GIVE RESPONSE in JSON FROM NOW ON IN CODEBLOCK. 
EVERY JSON RESPONSE SHOULD HAVE MUST FIELD understood: boolean like below if you want to give an acknowlegement
May be lets add your comments, it need not to be exactly like below the comments I think you understood
{
    json.dumps({
        "understood": True,
        "comments": "got it. you could provide with more vapps and its commands. I will help"
    })
}

Obviously if you have understood right. 
"""
gpt_response = gptSession.ask({
    "value" : json.dumps(prompt)
})
gpt_resp_dict = json.loads(gpt_response)
gpt_resp_dict

{'understood': True,
 'comments': 'Got it. You could provide me with more Vapps and their commands. I will be able to assist with those as well.'}

In [134]:
prompt= f"""
    Given below is the csv data of all elements that be switched using tab.
    {page_data}    
    Task: Apply for jobs in this linkedin page. I will get page related information and you give me the commands so that I can go forward.
    Remmeber it could be sign in , job search page, finally I need to submit a job application. If there is job we are trying to look into the add job_id field to response. 
    Now tell me what commands I need to execute to next step. USE JSON FORMAT AND FIELD NAMES THAT YOU INSTRUCTED TO DO
    TASK REQURIED INFORMATION:
    Credentials
    Username : sarath.devarakonda22@gmail.com
    password: 412,./dec
    
    IF THE PAGE DATA IS ALREDY SIGN IN NO NEED TO GO TO URL, BASED ON PAGE TYPE YOU GIVE COMMANDS
    ADD SLEEP of 1 COMMAND AFTER EVERY COMMAND
"""
gpt_response = gptSession.ask({
    "value" : json.dumps(prompt)
})
gpt_resp_dict = json.loads(gpt_response)
gpt_resp_dict

{'understood': True,
 'commands': [['send_keys',
   '/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV/DIV/DIV/INPUT',
   'sarath.devarakonda22@gmail.com'],
  ['sleep', 1],
  ['send_keys',
   '/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV[2]/DIV/DIV/INPUT',
   '412,./dec'],
  ['sleep', 1],
  ['click', '/HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV[2]/BUTTON'],
  ['sleep', 1],
  ['click', '/HTML/BODY/NAV/UL/LI[4]/A'],
  ['sleep', 1]]}

In [135]:
chrome_vapp.execute_commands(gpt_resp_dict["commands"])

Sent keys 'sarath.devarakonda22@gmail.com' to element with XPath: /HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV/DIV/DIV/INPUT
Sent keys '412,./dec' to element with XPath: /HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV/DIV[2]/DIV/DIV/INPUT
Clicked element with XPath: /HTML/BODY/MAIN/SECTION/DIV/DIV/FORM/DIV[2]/BUTTON
exception at Message: no such element: Unable to locate element: {"method":"xpath","selector":"/HTML/BODY/NAV/UL/LI[4]/A"}
  (Session info: chrome=122.0.6261.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104b14580 chromedriver + 3966336
1   chromedriver                        0x0000000104b0cb0c chromedriver + 3934988
2   chromedriver                        0x000000010478fda0 chromedriver + 277920
3   chromedriver                        0x00000001047d2394 chromedriver + 549780
4   chromedriver                        0x0

In [142]:
prompt = f"""
    Now the page data of specific elements given
    {specific_page_data}
    What to do and what are commands now
"""
gpt_response = gptSession.ask({
    "value" : json.dumps(prompt)
})
gpt_resp_dict = json.loads(gpt_response)
gpt_resp_dict

KeyboardInterrupt: 